<a href="https://colab.research.google.com/github/DelmiroDaladier/NLP-studies/blob/master/machine_translation/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import re
import numpy as np
import pandas as pd
from string import digits
from sklearn.model_selection import train_test_split

from keras import Model
from keras.models import load_model
from keras.layers import Dropout, Input, Embedding, LSTM, Dense

In [22]:
en_dataset = pd.read_csv('small_vocab_en.csv', header=None, error_bad_lines=False, sep='/n', nrows=15000)
fr_dataset = pd.read_csv('small_vocab_fr.csv', header=None, error_bad_lines=False, sep='/n', nrows=15000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [23]:
en_dataset.rename({0:'text'}, inplace=True, axis=1)
fr_dataset.rename({0:'text'}, inplace=True, axis=1)


In [24]:
remove_digits =  str.maketrans('', '', digits) 

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.lower())
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.lower())

en_dataset['text'] = en_dataset['text'].apply(lambda x: re.sub("[^\w\s]", "",x))
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: re.sub("[^\w\s]", "",x))

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.translate(remove_digits))
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.translate(remove_digits))

en_dataset['text'] = en_dataset['text'].apply(lambda x: x.strip())
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: x.strip())

en_dataset['text'] = en_dataset['text'].apply(lambda x: '<SOS> ' + x + ' <EOS>')
fr_dataset['text'] = fr_dataset['text'].apply(lambda x: '<SOS> ' + x + ' <EOS>')

In [25]:
word_to_index_en = {}
index_to_word_en = {}
word_index = 0
for sentence in en_dataset['text'].values:
  for word in sentence.split():
    if word not in word_to_index_en.keys():
      word_to_index_en[word] = word_index
      index_to_word_en[word_index] = word
      word_index += 1


In [26]:
max_len_en = max([len(sentence) for sentence in en_dataset['text']])

In [27]:
word_to_index_fr = {}
index_to_word_fr = {}
word_index = 0
for sentence in fr_dataset['text'].values:
  for word in sentence.split():
    if word not in word_to_index_fr.keys():
      word_to_index_fr[word] = word_index
      index_to_word_fr[word_index] = word
      word_index += 1

In [28]:
max_len_fr = max([len(sentence) for sentence in fr_dataset['text']])

In [29]:
num_encoder_tokens = len(word_to_index_en.keys())
num_decoder_tokens = len(word_to_index_fr.keys())

In [30]:
X = en_dataset['text'].values
Y = fr_dataset['text'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

In [56]:
def create_model_data (X, Y):
  encoder_input = np.zeros((X.shape[0], max_len_en, num_encoder_tokens), dtype='float32')
  decoder_input = np.zeros((X.shape[0], max_len_fr, num_decoder_tokens), dtype='float32')
  decoder_target = np.zeros((X.shape[0], max_len_fr, num_decoder_tokens), dtype='float32')

  for j, (input_sentence, target_sentence) in enumerate(zip(X, Y)):
      for pos, word in enumerate(input_sentence.split()):
        encoder_input[j, pos, word_to_index_en[word]] = 1.0
      encoder_input[j, pos+1, word_to_index_en['<EOS>']] = 1.0

      for pos, word in enumerate(target_sentence.split()):
        decoder_input[j, pos, word_to_index_fr[word]] = 1.0

        if pos > 0:
          decoder_target[j, pos-1, word_to_index_fr[word]] = 1.0

      decoder_input[j, pos, word_to_index_en['<EOS>']] = 1.0
      decoder_target[j, pos:, word_to_index_en['<EOS>']] = 1.0

  return encoder_input, decoder_input, decoder_target

In [57]:
encoder_input, decoder_input, decoder_target = create_model_data(X_train, Y_train)
encoder_input_test, decoder_input_test, decoder_target_test = create_model_data(X_test, Y_test)

In [12]:
latent_dim=64
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
model.compile(
    optimizer="rmsprop", loss='categorical_crossentropy', metrics=['accuracy']
)

model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
)

# Save model
model.save("s2s")

Epoch 1/100
169/169 [==============================] - 66s 388ms/step - loss: 0.8455 - accuracy: 0.8883 - val_loss: 0.4518 - val_accuracy: 0.8980
Epoch 2/100
169/169 [==============================] - 65s 382ms/step - loss: 0.4057 - accuracy: 0.9125 - val_loss: 0.3533 - val_accuracy: 0.9199
Epoch 3/100
169/169 [==============================] - 65s 383ms/step - loss: 0.3083 - accuracy: 0.9276 - val_loss: 0.2635 - val_accuracy: 0.9344
Epoch 4/100
169/169 [==============================] - 64s 380ms/step - loss: 0.2383 - accuracy: 0.9388 - val_loss: 0.2128 - val_accuracy: 0.9432
Epoch 5/100
169/169 [==============================] - 65s 383ms/step - loss: 0.1967 - accuracy: 0.9453 - val_loss: 0.1806 - val_accuracy: 0.9471
Epoch 6/100
169/169 [==============================] - 67s 394ms/step - loss: 0.1734 - accuracy: 0.9482 - val_loss: 0.1775 - val_accuracy: 0.9484
Epoch 7/100
169/169 [==============================] - 64s 379ms/step - loss: 0.1617 - accuracy: 0.9501 - val_loss: 0.1524 -

In [41]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


In [53]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))

    target_seq[0, 0, word_to_index_fr["<SOS>"]] = 1.0


    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_word_fr[sampled_token_index]
        decoded_sentence += sampled_char+" "

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "<EOS>" or len(decoded_sentence) > max_len_fr:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [58]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", X_test[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Real sentence:", Y_test[seq_index])

-
Input sentence: <SOS> the banana is their least favorite fruit  but the grape is your least favorite <EOS>
Decoded sentence: la banane est son fruit préféré moins mais la pomme est leur moins préférée <EOS> 
Real sentence: <SOS> la banane est leur fruit préféré moins  mais le raisin est votre préféré moins <EOS>
-
Input sentence: <SOS> china is usually wet during september  and it is usually pleasant in january <EOS>
Decoded sentence: chine est généralement humide en mars et il est généralement agréable en janvier <EOS> 
Real sentence: <SOS> chine est généralement humide en septembre  et il est généralement agréable en janvier <EOS>
-
Input sentence: <SOS> she dislikes grapefruit and mangoes <EOS>
Decoded sentence: elle déteste le pamplemousse et les raisins <EOS> 
Real sentence: <SOS> elle déteste le pamplemousse et les mangues <EOS>
-
Input sentence: <SOS> france is sometimes wet during september  but it is sometimes relaxing in spring <EOS>
Decoded sentence: la france est parfois 